# 🧠 O-ISAC CoT Master Pipeline

Tek notebook ile tüm extraction pipeline'ı çalıştır.

**Aşamalar:**
1. 📦 Setup & Mount
2. 🏭 Phase 1: Data Prep (PDF → Markdown)
3. 🖼️ Phase 2: Visual Analysis
4. 🧠 Phase 3: CoT Extraction
5. 📊 Results & Export

**Gereksinimler:**
- Colab GPU Runtime (T4 veya A100)
- GROQ_API_KEY (Colab Secrets'da ayarlı)

---
**Son Güncelleme:** 2025-12-11
**Versiyon:** 1.0

---
## 📦 Section 1: Setup & Mount

In [1]:
# @title 1.1 Install Dependencies
# Phase 1 & 2 heavy dependencies
!pip install marker-pdf -q
!pip install transformers torch pillow -q

# Phase 3 light dependencies
!pip install groq nest_asyncio pandas pyyaml -q
!pip install -q -U google-generativeai

print("✅ Tüm bağımlılıklar yüklendi!")

✅ Tüm bağımlılıklar yüklendi!


In [2]:
# @title 1.2 Mount Google Drive & Setup Paths
from google.colab import drive
from google.colab import userdata
import os
import sys

# Mount Drive
drive.mount('/content/drive')

# Project Paths
PROJECT_ROOT = '/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST'
NOTEBOOKS_DIR = os.path.join(PROJECT_ROOT, 'analysis/notebooks')
COT_LAB_DIR = os.path.join(PROJECT_ROOT, 'analysis/cot_laboratory')
PDF_DIR = os.path.join(PROJECT_ROOT, 'data/retrieved_docs')
MARKDOWN_DIR = os.path.join(PROJECT_ROOT, 'data/processed_markdowns')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'data/extraction_results_v3')

# Add to Python Path
sys.path.insert(0, NOTEBOOKS_DIR)
sys.path.insert(0, PROJECT_ROOT)

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📄 PDF Directory: {PDF_DIR}")
print(f"📝 Markdown Directory: {MARKDOWN_DIR}")
print(f"📊 Output Directory: {OUTPUT_DIR}")
print("✅ Paths configured!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Project Root: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST
📄 PDF Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/retrieved_docs
📝 Markdown Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/processed_markdowns
📊 Output Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/extraction_results_v3
✅ Paths configured!


In [3]:
# @title 1.3 Load API Key
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ GROQ_API_KEY yüklendi!")
except Exception as e:
    print("❌ HATA: Sol menüden 🔑 Secrets bölümüne GROQ_API_KEY ekleyin!")
    print(f"   Hata detayı: {e}")

✅ GROQ_API_KEY yüklendi!


---
## 🏭 Section 2: Phase 1 - Data Prep (PDF → Markdown)

**⚠️ GPU Gerektirir!** Bu adım PDF'leri OCR ile markdown'a çevirir.

In [4]:
# @title 2.1 Import Pipeline & Check Status
from extraction_pipeline_v3 import Config, CheckpointManager, phase1_marker_conversion

# Initialize
Config.init_dirs()
checkpoint = CheckpointManager(Config.CHECKPOINT_FILE)

# Show current status
processed = checkpoint.data.get('processed', {})
print(f"📊 Mevcut durum: {len(processed)} paper işlenmiş")
print(f"📂 PDF'ler: {PDF_DIR}")

# List PDFs
import glob
pdfs = glob.glob(os.path.join(PDF_DIR, '*.pdf'))
print(f"📄 Toplam PDF: {len(pdfs)}")

🌍 Environment: Google Colab
📊 Mevcut durum: 32 paper işlenmiş
📂 PDF'ler: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/retrieved_docs
📄 Toplam PDF: 32


In [ ]:
# @title 2.2 Run PDF → Markdown Conversion (Phase 1)
# ⚠️ Bu adım uzun sürebilir (paper başına ~1-2 dk)

print("⏳ Phase 1: PDF → Markdown dönüşümü başlıyor...")
phase1_marker_conversion(checkpoint, force_all=False)
print("✅ Phase 1 tamamlandı!")

⏳ Phase 1: PDF → Markdown dönüşümü başlıyor...

📄 PHASE 1: PDF → MARKDOWN (Marker)
Found 32 PDFs
   ⏩ O_ISAC_001 - already processed, skipping
   ⏩ O_ISAC_002 - already processed, skipping
   ⏩ O_ISAC_003 - already processed, skipping
   ⏩ O_ISAC_004 - already processed, skipping
   ⏩ O_ISAC_005 - already processed, skipping
   ⏩ O_ISAC_006 - already processed, skipping
   ⏩ O_ISAC_007 - already processed, skipping
   ⏩ O_ISAC_008 - already processed, skipping
   ⏩ O_ISAC_009 - already processed, skipping
   ⏩ O_ISAC_010 - already processed, skipping
   ⏩ O_ISAC_011 - already processed, skipping
   ⏩ O_ISAC_012 - already processed, skipping
   ⏩ O_ISAC_013 - already processed, skipping
   ⏩ O_ISAC_014 - already processed, skipping
   ⏩ O_ISAC_015 - already processed, skipping
   ⏩ O_ISAC_016 - already processed, skipping
   ⏩ O_ISAC_017 - already processed, skipping
   ⏩ O_ISAC_018 - already processed, skipping
   ⏩ O_ISAC_019 - already processed, skipping
   ⏩ O_ISAC_020 - already pro

---
## 🖼️ Section 3: Phase 2 - Visual Analysis

Gemini 1.5 Flash ve Gemini 1.5 Flash modelleri ile görsel analiz yapılır.

In [7]:
# @title 3.1 Run Visual Analysis (Phase 2)
from extraction_pipeline_v3 import phase2_visual_analysis

print("⏳ Phase 2: Görsel analiz başlıyor...")
phase2_visual_analysis(checkpoint)
print("✅ Phase 2 tamamlandı!")

⏳ Phase 2: Görsel analiz başlıyor...

👁️ PHASE 2: VISUAL ANALYSIS (Gemini 1.5 Flash - BATCHED)
Papers to analyze: 32
   ⏩ O_ISAC_001 - already analyzed, skipping
   ⏩ O_ISAC_002 - already analyzed, skipping
   ⏩ O_ISAC_003 - already analyzed, skipping
   ⏩ O_ISAC_004 - already analyzed, skipping
   ⏩ O_ISAC_005 - already analyzed, skipping
   ⏩ O_ISAC_006 - already analyzed, skipping
   ⏩ O_ISAC_007 - already analyzed, skipping
   ⏩ O_ISAC_008 - already analyzed, skipping
   ⏩ O_ISAC_009 - already analyzed, skipping
   ⏩ O_ISAC_010 - already analyzed, skipping
   ⏩ O_ISAC_011 - already analyzed, skipping
   ⏩ O_ISAC_012 - already analyzed, skipping
   ⏩ O_ISAC_013 - already analyzed, skipping
   ⏩ O_ISAC_014 - already analyzed, skipping
   ⏩ O_ISAC_015 - already analyzed, skipping
   ⏩ O_ISAC_016 - already analyzed, skipping
   ⏩ O_ISAC_017 - already analyzed, skipping
   ⏩ O_ISAC_018 - already analyzed, skipping
   ⏩ O_ISAC_019 - already analyzed, skipping
   ⏩ O_ISAC_020 - already an

---
## 🧠 Section 4: Phase 3 - CoT Extraction

Chain-of-Thought extraction ile yapısal veri çıkarma.

In [8]:
# @title 4.1 Import CoT Laboratory
sys.path.insert(0, COT_LAB_DIR)
from core.assembler import CoTAssembler
from core.batch_runner import CoTFactory

# Default Recipe
RECIPE_PATH = 'analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml'

print("✅ CoT Laboratory yüklendi!")
print(f"📜 Recipe: {RECIPE_PATH}")

✅ CoT Laboratory yüklendi!
📜 Recipe: analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml


In [9]:
# @title 4.2 Single Paper Test
# Önce tek bir paper üzerinde test et

TEST_PAPER_ID = "O_ISAC_029"  # @param {type:"string"}

import json

# Find paper markdown
paper_path = os.path.join(MARKDOWN_DIR, TEST_PAPER_ID, TEST_PAPER_ID, f"{TEST_PAPER_ID}.md")
vis_path = os.path.join(MARKDOWN_DIR, TEST_PAPER_ID, TEST_PAPER_ID, "visual_analysis.txt")

print(f"📄 Paper: {paper_path}")
print(f"   Exists: {os.path.exists(paper_path)}")

# Read content
with open(paper_path, 'r', encoding='utf-8') as f:
    content = f.read()

# Read visual if exists
visual_content = None
if os.path.exists(vis_path):
    with open(vis_path, 'r', encoding='utf-8') as f:
        visual_content = f.read()
    print(f"🖼️ Visual Analysis: {len(visual_content)} chars")

# Run extraction
assembler = CoTAssembler(PROJECT_ROOT)
result = assembler.run_extraction(
    RECIPE_PATH,
    content,
    paper_id=TEST_PAPER_ID,
    visual_content=visual_content
)

# Show result
if result['status'] == 'success':
    print("\n✅ EXTRACTION BAŞARILI!")
    print("\n📋 Reasoning Trace:")
    trace = result['parsed_output'].get('reasoning_trace', [])
    for step in trace:
        print(f"  {step.get('key')}: {step.get('value')[:80]}...")
else:
    print(f"\n❌ HATA: {result.get('error_message')}")

📄 Paper: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/processed_markdowns/O_ISAC_029/O_ISAC_029/O_ISAC_029.md
   Exists: True
🖼️ Visual Analysis: 15810 chars
[INFO] Loading Recipe: analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml...
[INFO] Assembling System Prompt from Modules...
[INFO] Calling Groq API (Model: llama-3.3-70b-versatile)...

[DEBUG] RAW RESPONSE LEN: 7302
[DEBUG] RAW RESPONSE START: {
  "reasoning_trace":[
      {
         "key":"step_0_visual_inspection",
         "type":"string",
         "required":true,
         "description":"MANDATORY: You MUST describe what you see in the ...
[INFO] Logging Run Evidence...
[OK] Run Logged: 20251211_214454_O_ISAC_029_llama-3.3-70b-versatile

✅ EXTRACTION BAŞARILI!

📋 Reasoning Trace:
  step_0_visual_inspection: The paper includes several figures and charts that illustrate the system archite...
  step_1_concept_analysis: The system uses a photonic-based THz ISAC architecture, where commun

In [ ]:
# @title 4.3 Batch Extraction (Tüm Paper'lar)
# ⚠️ Bu uzun sürecek! ~2-3 dk per paper

RUN_BATCH = False  # @param {type:"boolean"}

if RUN_BATCH:
    factory = CoTFactory(PROJECT_ROOT)
    factory.run_batch(RECIPE_PATH)
else:
    print("ℹ️ Batch mode kapalı. Çalıştırmak için RUN_BATCH = True yapın.")

ℹ️ Batch mode kapalı. Çalıştırmak için RUN_BATCH = True yapın.


---
## 📊 Section 5: Results & Export

In [ ]:
# @title 5.1 View Latest Logs
import glob
from datetime import datetime

logs_dir = os.path.join(COT_LAB_DIR, 'logs')
log_files = sorted(glob.glob(os.path.join(logs_dir, '*_RESULT.json')))[-5:]

print(f"📋 Son 5 extraction log:")
for log in log_files:
    filename = os.path.basename(log)
    # Parse: 20251211_093015_O_ISAC_029_llama-3.3-70b-versatile_RESULT.json
    parts = filename.split('_')
    date_str = parts[0]  # YYYYMMDD
    time_str = parts[1]  # HHMMSS
    paper_id = f"{parts[2]}_{parts[3]}_{parts[4]}"

    # Format full timestamp
    try:
        dt = datetime.strptime(f"{date_str}_{time_str}", "%Y%m%d_%H%M%S")
        formatted = dt.strftime("%Y-%m-%d %H:%M:%S")
    except:
        formatted = f"{date_str}_{time_str}"

    print(f"  • {formatted} | {paper_id}")

In [ ]:
# @title 5.2 Export to CSV (TODO)
# Bu fonksiyon tüm log JSON'larını birleştirip CSV'ye çevirecek

print("📊 CSV export fonksiyonu henüz implemente edilmedi.")
print("   Sonuçlar logs/ klasöründe JSON olarak mevcut.")

📊 CSV export fonksiyonu henüz implemente edilmedi.
   Sonuçlar logs/ klasöründe JSON olarak mevcut.


---
## ✅ Done!

**Sonraki Adımlar:**
1. Single paper test sonuçlarını kontrol et
2. Kalite iyi ise batch mode'u aç
3. Tüm paper'ları işle
4. CSV export yap

In [6]:
!rm "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/processed_markdowns/O_ISAC_029/O_ISAC_029/visual_analysis.txt"

In [10]:
!cat "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/processed_markdowns/O_ISAC_029/O_ISAC_029/visual_analysis.txt"

Image [_page_10_Figure_2.jpeg]: Chart.
(a) **BER vs Center Frequency (GHz):** This chart shows the Bit Error Rate (BER) as a function of the center frequency. The BER decreases from approximately 1.8E-2 at 118 GHz to a minimum of 1.0E-3 at 132 GHz, then increases back to approximately 1.8E-2 at 142 GHz. The insets display electrical spectra at 118 GHz and 142 GHz, illustrating the modulated signal.
(b) **BER vs Vpp (mV):** This chart plots BER against the peak-to-peak voltage (Vpp). The BER starts at approximately 2.0E-2 at 400 mV, reaches a minimum of 1.0E-3 at 600 mV, and then rises to approximately 6.0E-3 at 900 mV. Insets show 16QAM constellation diagrams at 400 mV and 600 mV.
(c) **BER vs Baudrate (GHz):** This chart illustrates the BER performance as the baudrate increases. The BER rises from approximately 1.5E-4 at 15 GHz to about 4.5E-3 at 30 GHz. Insets show a 16QAM constellation diagram at 15 GHz and an electrical spectrum at 30 GHz.
(d) **BER vs Fiberlength (km):** This bar 